In [ ]:
!pip install -q apache_beam[gcp]

!pip install -q stop_words

!wget -q "https://github.com/Internite/Intra/raw/notebook/Helper.py"

!wget -q "https://github.com/Internite/Intra/raw/notebook/Heline.py"

!wget -q "https://github.com/Internite/Hantu/raw/notebook/Histplot.py"

In [2]:
import tensorflow as tf

import tensorflow_datasets as tfds

from Helper import stopper

tfds.disable_progress_bar()

In [ ]:
data = "imdb_reviews"; true = True

(train, testi), info = tfds.load(data, split=["train", "test"], as_supervised=true, with_info=true)

In [4]:
info.splits["train"].num_examples, info.splits["test"].num_examples

(25000, 25000)

In [5]:
train_input = []
train_label = []

testi_input = []
testi_label = []

for data, label in train:
  train_input.append(stopper(data.numpy().decode("utf8")))
  train_label.append(label.numpy())

for data, label in testi:
  testi_input.append(stopper(data.numpy().decode("utf8")))
  testi_label.append(label.numpy())

In [6]:
len(train_input), len(train_label)

(25000, 25000)

In [7]:
len(testi_input), len(testi_label)

(25000, 25000)

In [ ]:
from tensorflow.keras import layers, Model

x = layers.Embedding()
x = layers.Bidirectional(layers.LSTM(64))
x = layers.GlobalAveragePooling1D()
x = layers.Dense(10, activation="relu")
output = layers.Dense(1, activation="sigmoid")

model = Model(outputs=output)

In [ ]:
metrics = "binary_accuracy"; loss = "binary_crossentropy"

model.compile(optimizer="adam", loss=loss, metrics=[metrics])

hist = model.fit(train, validation_data=testi)

In [ ]:
model.save("/tmp/model")

In [ ]:
from Histplot import Histplot

histplot = Histplot(hist)

histplot.Trainplot()

In [ ]:
histplot.Testiplot()

In [ ]:
!pip install -q tensorflowjs

In [ ]:
import apache_beam as B

from Helper import Modeli

from apache_beam.ml.inference.base import RunInference
from apache_beam.ml.inference.base import KeyedModelHandler
from apache_beam.ml.inference.tensorflow_inference import TFModelHandlerTensor
from apache_beam.options.pipeline_options import PipelineOptions

model_handler = TFModelHandlerTensor("/tmp/model")

trainer = [train[10]]

with B.Pipeline() as pipeline:
    runner = (
        pipeline
        | "Create" >> B.Create(trainer)
        | "Modeler" >> RunInference(model_handler)
        | "Perform" >> B.ParDo(Modeli())
        | "Final" >> B.Map(print)
    )